### Import General Python libs and modules

In [2]:
import sys
sys.path.insert(0, "../")

from datetime import datetime, timedelta
from pprint import pprint
from pathlib import Path
import pandas as pd

### Import feast related modules and definitions from local directories

In [3]:
from feast import FeatureStore
from entities.entity import driver
from features.feature_views import driver_hourly_stats_view
from fservice.feature_svc import driver_fs
from train import get_training_data
from fvector import get_feature_vector

/usr/local/lib/python3.9/site-packages/feast/feature_view.py:97: DeprecationWarning: The argument 'input' is being deprecated. Please use 'batch_source' instead. Feast 0.13 and onwards will not support the argument 'input'.
  warnings.warn(


#### Define repo path and create a Feature Store instance

In [4]:
FEAST_REPO = "/Users/jules/git-repos/feast_workshops/module_1/feature_repo/"
repo_path = Path(FEAST_REPO)
fs = FeatureStore(repo_path=repo_path)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Step 1. Register the data source, entity, features in the FeatureView, and the FeatureService with the Feast Registry

In [5]:
fs.apply([driver, driver_fs, driver_hourly_stats_view])

#### Get the training historical data

In [6]:
training_df = get_training_data(repo_path)
training_df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips
0,2021-04-12 08:12:10+00:00,1002,0.121337,0.205787,400
1,2021-04-12 10:59:42+00:00,1001,0.156569,0.297756,860
2,2021-04-12 15:01:12+00:00,1004,0.122386,0.868674,80
3,2021-04-12 16:40:26+00:00,1003,0.905787,0.853927,848


### Step 2. Now materialize, load data from offline into online store

In [7]:
fs.materialize_incremental(end_date=datetime.utcnow() - timedelta(minutes=0))

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Materializing 1 feature views to 2021-08-12 18:11:44-07:00 into the sqlite online store.

driver_hourly_stats from 2020-08-12 18:11:44-07:00 to 2021-08-12 18:11:44-07:00:


100%|███████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1088.86it/s]


### Step 3: Get the feature vector for inference from the online store

In [8]:
# get the feature vector for inferencing from the online store
for driver in [1001, 1002, 1003]:
    feature_vector = get_feature_vector(repo_path, driver)
    print("--" * 5)
    pprint(feature_vector)

----------
   acc_rate  avg_daily_trips  conv_rate  driver_id
0  0.288092              679   0.522771       1001
----------
   driver_id  acc_rate  conv_rate  avg_daily_trips
0       1002  0.180588   0.393899               55
----------
   acc_rate  driver_id  avg_daily_trips  conv_rate
0  0.717372       1003               39   0.900113
